In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:


storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)

alphas = [
    'ES_ContFut_Long_DMI_exit_on_indicator__Bullish_0_Feb02_custom',
    'ES_ContFut_Long_DMI_exit_on_indicator__Bullish_1_Feb02_custom',
    'ES_ContFut_Long_Strategy_Seasonality_DailySeasTracking__Seasonal_Bullish_0_Feb_20_custom',
]


In [ ]:
dm = DataManager()
dm.session_set('US.ES')
dm.series_primary_set(QuoteContFut, 'US.ES', timeframe='D')


In [ ]:
swarms = storage.swarms_data( alphas)
exposure_df = pd.DataFrame({k: v['swarm_series']['exposure'] for k,v in swarms.items()})

In [ ]:
px = dm.quotes()

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrfeed.quotes.quote_contfut import QuoteContFut
from tmqrfeed.costs import Costs
from datetime import datetime
import pandas as pd
from tmqrstrategy import StrategyBase
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic
from tmqrfeed.quotes import QuoteIndex

# 
# Using some of V1 libs
#
from exobuilder.data.exostorage import EXOStorage
from scripts.settings import *



class AlphaRiskReversalHedge(StrategyBase):
    def __init__(self, datamanager: DataManager, **kwargs):
        super().__init__(datamanager, **kwargs)


    def setup(self):
        swarms = storage.swarms_data(self.context['v1_alphas'])
        self.exposure_df = pd.DataFrame({k: v['swarm_series']['exposure'] for k,v in swarms.items()})
        
        
        self.dm.session_set(self.context['instrument'])
        
        self.dm.series_primary_set(QuoteContFut, self.context['instrument'],
                                   timeframe='D')        
        
        HEDGE_IDX_NAME = self.context['index_hedge_name']        
        self.dm.series_extra_set('index_hedge', QuoteIndex, HEDGE_IDX_NAME, set_session=True, check_session=True)
        
        self.dm.costs_set('US', Costs())

    def calculate(self, *args):
        direction, = args

        # Defining EXO price
        px = self.dm.quotes()['c']
        
        #
        # Calculating exposure of V1 alphas
        #
        exposure = self.exposure_df.sum(axis=1)
        exposure.index = exposure.index.tz_localize(px.index.tz)
        exposure = exposure.reindex(px.index, method='ffill')
        
        return pd.DataFrame({'exposure': exposure})

    def calculate_position(self, date: datetime, exposure_record: pd.DataFrame):
        primary_quotes_position = self.dm.position()
                

        # get net exposure for all members
        exposure = exposure_record['exposure'].sum()
        
        #
        # Do new position management magic here
        #
        index_hedge_position = self.dm.position('index_hedge')
        try:
            hedge_position_rec = index_hedge_position.get_net_position(date)
            # Add index position as hedge
            # NOTE: exposure - is a alpha's exporure of trade, when alpha is out of market
            #                  exposure equals 0, this means that means no position and hedge
            # NOTE: self.context['index_hedge_direction'] allowed 1, -1, or even 0 - i.e. no hedge
            self.position.add_net_position(date, hedge_position_rec, qty=abs(exposure)*self.context['index_hedge_direction'])
        except PositionNotFoundError as exc:
            log.error(f"Couldn't find hedged index position! {exc}")


In [ ]:
import logging
from tmqr.logs import log
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)

In [ ]:
dm = DataManager()

ALPHA_CONTEXT = {
    'name': 'AlphaV1Exposure_HedgedBy_V2_Index',
    'wfo_params': {
        'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
        'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
        'oos_periods': 2,  # Number of months is OOS period
        'iis_periods': 12,  # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
    },
    'wfo_optimizer_class': OptimizerBase,
    'wfo_optimizer_class_kwargs': {
        'nbest_count': 1,
        'nbest_fitness_method': 'max'
    },
    'wfo_opt_params': [
        ('direction', [1]),
    ],
    'wfo_members_count': 1,
    'wfo_costs_per_contract': 0.0,
    'wfo_scoring_type': 'netprofit',
    
    
    'context': {
        'index_hedge_name': 'US.CL_EXOSemiFuture_Delta25', # Name of the index used for hedge
        'index_hedge_direction': 1, # ALLOWED 1, -1, or even 0 - i.e. no hedge
        
        'instrument': 'US.CL',
        'v1_alphas': [
            'CL_ContFut_Long_IchimokuCloud__Bearish_Jan17_2custom',
            'CL_ContFut_Long_IchimokuCloud__Bearish_Jan17custom',
            'CL_ContFut_Long_IchimokuCloud__Bullish_Jan17_2custom',
            'CL_ContFut_Long_IchimokuCloud__Bullish_Jan17custom',
        ]
    }
}

alpha = AlphaRiskReversalHedge(dm, **ALPHA_CONTEXT)

alpha.run()

In [ ]:
equity = alpha.position.get_pnl_series()

In [ ]:
equity.equity_decision.plot()